---
__<center>The included code showcases the methods that have been used, although it doesn't generate all the data found in the report.</center>__

---

# Ground state of the transverse Ising model with VQE

The transverse Ising model (TIM) is defined by the following qubit hamiltonian:
$$
\hat{H} = -J\sum_{\langle i,j\rangle}\sigma_{i}^{z}\sigma_{j}^{z} - B\sum_{i}\sigma_{i}^{x}.
$$
where $J=1$ is the coupling and $B$ is the external transverse magnetic field. I will assume that $B=\frac{1}{4}$.

In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from matplotlib import pyplot as plt
from qiskit_nature.second_q.hamiltonians import IsingModel
from qiskit_nature.second_q.hamiltonians.lattices import BoundaryCondition, HyperCubicLattice, KagomeLattice, LineLattice
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import Operator
from qiskit.visualization import array_to_latex

#line_tim = qml.data.load("qspin", sysname="Ising", periodicity="open", lattice="chain", layout="1x16")[0]

Load and save your IBMQ credentials if needed before continuing.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Estimator

service = QiskitRuntimeService(name="Panagiotis Constantinidis")
backend = service.get_backend("ibmq_qasm_simulator")#noise-less simulator on the cloud

options = Options()
options.execution.init_qunits = True # initialize qubits to |0>
options.execution.shots = 1000# number of shots per experiment

## Hypercubic lattice

Create the lattice and the Hamiltonian in an `Operator` form, in order for the estimator primitive to be able to evaluate it. Additionaly, I instantiate the optimizer and the ansatz.

In [2]:
N = 2
J = 1
B = 0.25
maxiter = 100
lattice = HyperCubicLattice(size=(N,N,N))
ising_matrix = IsingModel(lattice.uniform_parameters(uniform_interaction=J,\
                                                        uniform_onsite_potential=B)).second_q_op().to_matrix()
ising_hamiltonian = Operator(ising_matrix)

optimizer = COBYLA(maxiter=maxiter, disp=True)
ansatz = RealAmplitudes(lattice.num_nodes, reps=3, entanglement='reverse_linear', flatten=True)

The initial parameters should be the same for all the algorithm implementations in order for them to be comparable.

In [ ]:
initial_params = pnp.array([0.62476071, 0.65856904, 0.85165179, 0.92671967, 0.68046261, 0.54199521,
                           0.07561732, 0.99096942, 0.84277395, 0.18476281, 0.95033405, 0.28702002,
                           0.30162193, 0.15901641, 0.31483114, 0.06107861, 0.57753898, 0.34316103,
                           0.17362444, 0.08495347, 0.36485795, 0.45544241, 0.8825014, 0.47694389,
                           0.46326857, 0.98616173, 0.90407303, 0.66730942, 0.5169122, 0.30944371,
                           0.70817102, 0.54354616])

ansatz.decompose().draw("mpl")

### Simulating on the IBMQ cloud
__Sessions__ in Qiskit runtime is the go-to way to implement iterative algorithms. In this case the optimizer will run up to `maxiter` quantum subroutines, by using a session these are treated as a single entity and *do not* enter the queue `maxiter` times. Using a context manager is advised as it is less error prone and more resource efficient.

In [ ]:
bounds = []
for _ in range(ansatz.num_parameters):
    bounds.append((-1, 1))

In [ ]:
with Session(service=service, backend=backend) as session:
        estimator = Estimator(session=session, backend=backend, options=options)
        cost_func = lambda params: estimator.run(ansatz, ising_hamiltonian, parameter_values=params).result().values[0]
        result = optimizer.minimize(cost_func, x0=initial_params, bounds=bounds)

In [ ]:
result.x

---
## *Pennylane* usecase

Additional reading: [Accelerating VQEs with quantum natural gradient](https://pennylane.ai/qml/demos/tutorial_vqe_qng/)

In [ ]:
ising_matrix = IsingModel(lattice.uniform_parameters(uniform_interaction=J,\
                                    uniform_onsite_potential=B)).second_q_op().to_matrix()
observable = qml.Hermitian(ising_matrix, wires=[0,1,2,3,4,5,6,7])
hamiltonian = qml.Hamiltonian((1,), (observable,))

In [ ]:
#initial_params = np.array(initial_params)
dev = qml.device("default.qubit", wires=lattice.num_nodes)

def real_amplitudes(params, wires=[0,1,2,3,4,5,6,7]):
    l = len(wires)
    def cx_layer():
        for i in wires[-2::-1]: # reverse linear
            qml.CNOT(wires=[i, i+1])
    
    def ry_layer(param_range:list):
        for i in range(param_range[0], param_range[1]+1):
            qml.RY(params[i], wires=i%l)
    
    ry_layer([0, l-1])
    cx_layer()
    ry_layer([l, 2*l-1])
    cx_layer()
    ry_layer([2*l, 3*l-1])
    cx_layer()
    ry_layer([3*l, 4*l-1])

In [ ]:
@qml.qnode(dev, interface="autograd")
def cubic_cost_function(params):
    real_amplitudes(params)
    return qml.expval(hamiltonians[0])

---
### Studying the phase transition of a 1D lattice

In [ ]:
l_lattice = LineLattice(10, boundary_condition=BoundaryCondition.PERIODIC)
fields = np.arange(-2, 3, 0.25)
l_ising_matrices = [IsingModel(l_lattice.uniform_parameters(uniform_interaction=J,\
                                                        uniform_onsite_potential=b)).second_q_op().to_matrix() for b in fields]

In [ ]:
observables = [qml.Hermitian(ising_matrix, wires=list(range(10))) for ising_matrix in l_ising_matrices]
hamiltonians = [qml.Hamiltonian((1,), (obs,)) for obs in observables]

In [ ]:
optimal_params = [pnp.random.rand(32)]
l_energies = []
l_exacts = []
paulix_exp = []
for h in l_ising_matrices:
    l_exacts.append(min(np.linalg.eigh(h)[0]))

In [ ]:
dev = qml.device("default.qubit", wires=10)
@qml.qnode(dev, interface="autograd")
def line_cost_function(params):
    real_amplitudes(params)
    return qml.expval(hamiltonians[19])

In [ ]:
parameters = vqe_cobyla(cubic_cost_function, theta0=optimal_params[-1], max_iterations=250)
hist = [cubic_cost_function(p) for p in parameters]
l_energies.append(hist[-1])
optimal_params.append(parameters[-1])

In [ ]:
@qml.qnode(dev, interface="autograd")
def pauliz_expectation(opt_params):
    obs = 0
    for i in range(10):
        obs += qml.PauliX(wires=i)
    real_amplitudes(opt_params)
    return qml.expval(obs)

In [ ]:
xexp = []
for p in optimal_params[:-1]:
    xexp.append(pauliz_expectation(p))

### COBYLA case

In [ ]:
def vqe_cobyla(objective_function, theta0=initial_params, max_iterations=400, bounds=bounds):
    params_history = [theta0]
    def callback(params):
        params_history.append(params)

    opt = COBYLA(maxiter=max_iterations, callback=callback)
    opt.minimize(objective_function, params_history[0], bounds=bounds)
    return params_history

In [ ]:
params = vqe_cobyla(cubic_cost_function)
energies = [cubic_cost_function(args) for args in params]

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(np.arange(1, len(energies)+1), energies)
plt.axhline(y=exact1,color="red",linestyle="-.", label="exact")
plt.grid()

---
### Quantum natural gradient descent (QNGD) case

In [ ]:
maxiter = 400
step_size = 0.14
conv_tol = 1e-7

In [ ]:
def vqe_qngd(objective_function, theta0, step_size=step_size, max_iterations=maxiter, conv_tol=conv_tol, disp_rate=50):
    opt = qml.QNGOptimizer(step_size, approx="block-diag")
    
    #params_history = [init_params]
    energy_callback = []
    params = theta0
    for n in range(max_iterations):
        params, prev_energy = opt.step_and_cost(objective_function, params)

        energy = objective_function(params)
        conver = np.abs(energy - prev_energy)
        energy_callback.append(prev_energy)

        if n % disp_rate == 0:
            print(f"Iteration = {n},  Energy = {energy}")

        if conver <= conv_tol:
            break
        #ground_energies.append(energy)
    print(f"Final value of the ground-state is: {energy}")
    return energy_callback, params

In [ ]:
initial_params = np.array(initial_params, requires_grad=True)
ng_energies, opt_params = vqe_qngd(cubic_cost_function, initial_params, disp_rate=50)

### Kagome lattice

In [ ]:
k_lattice = KagomeLattice(2, 2, boundary_condition=BoundaryCondition.PERIODIC).uniform_parameters(J, B)
k_ising_matrix = IsingModel(k_lattice).second_q_op().to_matrix()
obs = qml.Hermitian(k_ising_matrix, wires=list(range(12)))
k_ising_hamiltonian = qml.Hamiltonian((1,), (obs,))

In [ ]:
def efficient_su2(params, wires=list(range(12))):
    l = len(wires)
    def cx_layer():
        for i in wires[-2::-1]: # reverse linear
            qml.CNOT(wires=[i, i+1])
    
    def ry_layer(param_range:list):
        for i in range(param_range[0], param_range[1]+1):
            qml.RY(params[i], wires=i%l)
            
    def rx_layer(param_range:list):
        for i in range(param_range[0], param_range[1]+1):
            qml.RZ(params[i], wires=i%l)
            
    ry_layer([0, 11])
    rx_layer([12, 23])
    cx_layer()
    ry_layer([24, 35])
    rx_layer([36, 47])
    cx_layer()
    ry_layer([48, 59])
    rx_layer([60, 71])
    cx_layer()
    ry_layer([72, 83])
    rx_layer([84, 95])

In [ ]:
dev = qml.device("default.qubit", wires=12)

@qml.qnode(dev, interface="autograd")
def kagome_cost_function(params):
    for i in range(12):
        if i%2 == 0:
            qml.PauliZ(wires=i)
    real_amplitudes(params)
    #efficient_su2(params)
    return qml.expval(k_ising_hamiltonian)

In [ ]:
init_params = np.random.rand(48)
#init_params_su2 = np.random.rand(96)

In [ ]:
k_params = vqe_cobyla(kagome_cost_function, max_iterations=600, theta0=init_params)

In [ ]:
for i, p in enumerate(k_params):
    if not isinstance(p, np.ndarray):
        print(i)

In [ ]:
k_energies = [kagome_cost_function(p) for p in k_params]

In [ ]:
k_ref_params = vqe_cobyla(kagome_cost_function, max_iterations=600, theta0=init_params)

In [ ]:
k_ref_energies = [kagome_cost_function(p) for p in k_ref_params]

In [ ]:
k_refz_params = vqe_cobyla(kagome_cost_function, max_iterations=600, theta0=init_params)

In [ ]:
k_refz_energies = [kagome_cost_function(p) for p in k_ref_params]

In [ ]:
values, vectors = np.linalg.eigh(k_ising_matrix)
ground_state = 0
for i, val in enumerate(values):
    if val == min(values):
        ground_state = vectors[:,i]
        break
k_exact = min(values)

In [ ]:
plt.figure(figsize=(9, 5))
plt.plot(range(1, len(k_energies)+1), k_energies, label="RealAmplitudes")
plt.plot(range(1, len(k_ref_energies)+1), k_ref_energies, label="RealAmplitudes $U_{R,X}$")
#plt.plot(range(1, len(k_refz_energies)+1), k_refz_energies, label="RealAmplitudes $U_{R,Z}$")
plt.axhline(y=k_exact,color="red",linestyle="-.", label="exact")
plt.legend(fancybox=False, edgecolor="black", fontsize="x-large")
plt.title("Kagome ground energy approximation")
plt.xlabel("Iterations", size=13)
plt.ylabel("Expextation valueσ", size=13)
plt.grid()

---
## Monitor results from IBM quantum service

Fill in your session ID which can be found on the jobs dashboard at the IBMQ website.

In [ ]:
jobs = service.jobs(session_id="cn2c4i7n6tkp35haqh50", limit=None)#cmj63dn4mi97k7k12sr0
variance = [job.result().metadata[0]["variance"] for job in jobs]
energies = [job.result().values[0] for job in jobs]

In [ ]:
values, vectors = np.linalg.eigh(ising_matrix)
ground_state = 0
for i, val in enumerate(values):
    if val == min(values):
        ground_state = vectors[:,i]
        break
exact1 = min(values)

### Ploting outline

In [ ]:
plt.figure(figsize=(9,5))
plt.plot(range(1, len(energies)+1), energies, label="COBYLA")
plt.plot(range(1, len(ng_energies[0])+1), ng_energies[0], label="QNGD", color='orange')
plt.axhline(y=exact,color="red",linestyle="-.", label="exact")
#plt.fill_between(range(1, len(energies)+1), np.array(energies[::-1])-np.array(variance[::-1]),\
#                 np.array(energies[::-1])+np.array(variance[::-1]), color="paleturquoise", label="variance")
plt.ylabel(r"Expectation value", size=12)
plt.xlabel(r"Iterations", size=13)
#plt.text(65, 2, r"$\left|\frac{E_{qngd} - E_{exact}}{E_{cobyla} - E_{exact}}\right| \approx $"+f"{round(np.abs(ng_energies[-1]-exact / energies[0]-exact)**-1, 4)}", bbox=dict(facecolor='white', edgecolor='black'), fontsize=15)

plt.grid()
plt.legend(fontsize="x-large", fancybox=False, edgecolor="black")
plt.title(r"$\langle E\rangle_{2\times 2\times 2}$ ground energy approximation")
print(f"nfev: {len(energies)}\nfval: {energies[0]}")